# Use Case: Predictive Situation Awareness for Ebola 

In [1]:
import random
import json
import folium
import folium.plugins as plugins
from pprint import pprint
from folium.plugins import HeatMap
import numpy as np
import pprint

In [2]:
def initAfricaMap():
    with open('Region_map.json') as f:
        data = json.load(f)
    return data

In [3]:
import requests 
import json

def createJsonRequest(mode, type, jsonData):
    json = "{\"mode\":\"" +  mode + "\", \"type\":\"" + type + "\",\"data\":" + jsonData + "}"
    return json

def RequestReasoning(ev, dz_dic = None):
    
    if dz_dic != None:
        for r in dz_dic.items(): 
            id = r[0].replace('NC','')
            mean_var = r[1]
            ev += "NewCase" + id + "=" + mean_var[0] + ";"
  
    body = createJsonRequest("update", "json", "{\"name\":\"EBOLA\", \"EV\":\"" + ev +  "\"}")

    r = requests.post('http://169.60.94.50:8080/BC/webapi/RequestBN/run', data = body)

    jdata = json.loads(r.text) 
    dz_dic = {}

    for d in jdata:
        for key, value in d.items(): 
            if key == "Name":
                if "NC_" in value: 
                    typeRV = d["type"]
                    param = d["Param"]
                    
                    if "bel_con" in typeRV:  
                        mean = param["Mean"]
                        variance = param["Variance"]
                        mean_var = [mean, variance]
                        dz_dic.update({value:mean_var})
                    elif "ev_con" in typeRV:  
                        mean = param["Evidence"]
                        variance = -1
                        mean_var = [mean, variance]
                        dz_dic.update({value:mean_var})

    return dz_dic
                
# print(dz_dic)

In [4]:
def RenderMap(data, dz_dic_list, makerShow = False): 
    # set folium Map   
    m = folium.Map([8.277222, -10.573889] , zoom_start=7)

    dataheatTime=[]
    for dz_dic in dz_dic_list: 
        dataheat=[]  
        
        for d in data:
#             print (d)
            dz_id = d['ID']
            place = dz_id + ":" + d['Name']
            lat = float(d['Lat'])
            lng = float(d['Lng'])

            # get reasoned data 
            mean_var = dz_dic.get(dz_id)  
            mean = mean_var[0]
            var = mean_var[1]
            
#             print (mean_var)
            
            # set maker and heatmap
            radius = 140*(float(mean))
            size = int(float(mean)/10)

            myd=[]
            myd1=[]
            myd1.append(lat)
            myd1.append(lng)
            myd1.append(size*10)
            myd.append(myd1)

#                print(myd1, size)
            for e in range(size):
                r_x = np.random.normal(0, 0.1, 1)
                r_y = np.random.normal(0, 0.1, 1)
#                             print(r_x, r_y)
                newDot = [myd1[0] + r_x[0], myd1[1] + r_y[0], myd1[2]]
                dataheat.append(newDot)
 
            popstr = place
    
            if var == -1:
                popstr += " ev: " + mean 
            else: 
                popstr += " M: " + mean + " V: " + var

            popstr += " S: " + str(size)

            if makerShow == True:
                folium.Marker(
                    icon=folium.Icon(color='green'),
                    location=[lat, lng], 
                    popup=folium.Popup(popstr, parse_html=True)
                ).add_to(m)      

        dataheatTime.append(dataheat)
        
    # HeatMap(dataheat,radius = 30).add_to(m) 
    plugins.HeatMapWithTime(dataheatTime, radius = 50).add_to(m) 

    return m
  

# gradient : dict, default None
#         Color gradient config. e.g. {0.4: 'blue', 0.65: 'lime', 1: 'red'}
# ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
#              'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
#              'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
#              'gray', 'black', 'lightgray']

In [5]:
def EbloaMapShow(ev, time, makerShow = False): 
    data = initAfricaMap()
    dz_dic_list = []
    dz_dic = None
    for t in range(time):
        dz_dic = RequestReasoning(ev, dz_dic)
#         pprint.pprint(dz_dic)
        dz_dic_list.append(dz_dic)
        
    m = RenderMap(data, dz_dic_list, makerShow)
    return m

## Case 1: Ebola cases were reported in Moyamba 

In [6]:
ev = "NewCase_5=1;"

m = EbloaMapShow(ev, 9)
m

In [7]:
ev = "NewCase_33=1;"

m = EbloaMapShow(ev, 9)
m